# Using Kipoi from R

Thanks to the [reticulate](https://github.com/rstudio/reticulate) R package from RStudio, it is possible to easily call python functions from R. Hence one can use kipoi python API from R. This tutorial will show how to do that.

## Install and load `reticulate`

Make sure you have the reticulate R package installed

In [6]:
# install.packages("reticulate")

In [2]:
library(reticulate)

## Setup the python environment

With `reticulate::py_config()` you can check if the python configuration used by reticulate is correct. You can can also choose to use a different conda environment with `use_condaenv(...)`. This comes handy when using different models depending on different conda environments.

In [3]:
reticulate::py_config()

python:         /opt/modules/i12g/anaconda/3-4.1.1/bin/python
libpython:      /opt/modules/i12g/anaconda/3-4.1.1/lib/libpython3.5m.so
pythonhome:     /opt/modules/i12g/anaconda/3-4.1.1:/opt/modules/i12g/anaconda/3-4.1.1
version:        3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06)  [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy:          /opt/modules/i12g/anaconda/3-4.1.1/lib/python3.5/site-packages/numpy
numpy_version:  1.13.3

python versions found: 
 /opt/modules/i12g/anaconda/3-4.1.1/bin/python
 /usr/bin/python

In [15]:
# reticulate::conda_list()

In [17]:
# reticulate::use_condaenv(...)

## Load kipoi

In [4]:
kipoi <- import("kipoi")

### List models

In [5]:
kipoi$list_models()$head()

  source                      model version  \
0  kipoi                 MaxEntScan     0.1   
1  kipoi              extended_coda     0.1   
2  kipoi                    DeepSEA     0.1   
3  kipoi                        HAL     0.1   
4  kipoi  CpGenie/MCF_7_ENCSR943EFS     0.1   

                                             authors  \
0  [Author(name='Jun Cheng', github='s6juncheng',...   
1  [Author(name='Johnny Israeli', github='jisrael...   
2  [Author(name='Lara Urban', github='LaraUrban',...   
3  [Author(name='Jun Cheng', github='s6juncheng',...   
4  [Author(name='Haoyang Zeng', github='haoyangz'...   

                                                 doc    type  \
0                              MaxEnt Splicing Model  custom   
1            Single bp resolution ChIP-seq denoising   keras   
2  This CNN is based on the DeepSEA model from Zh...   keras   
3                      Model from Rosenberg ... TODO  custom   
4  Abstract: DNA methylation plays a crucial role...   keras

`reticulate` currently doesn't support direct convertion from `pandas.DataFrame` to R's `data.frame`. Let's make a convenience function to create an R dataframe via matrix conversion.

In [6]:
#' List models as an R data.frame
kipoi_list_models <- function() {
    df_models <- kipoi$list_models()
    df <- data.frame(df_models$as_matrix())
    colnames(df) = df_models$columns$tolist()
    return(df)
   
}

In [7]:
df <- kipoi_list_models()

In [8]:
head(df, 2)

source,model,version,authors,doc,type,inputs,targets,tags
kipoi,MaxEntScan,0.1,"[Author(name='Jun Cheng', github='s6juncheng', email=None)]",MaxEnt Splicing Model,custom,seq,psi,<environment: 0x2b762e8>
kipoi,extended_coda,0.1,"[Author(name='Johnny Israeli', github='jisraeli', email=None)]",Single bp resolution ChIP-seq denoising,keras,H3K27AC_subsampled,H3K27ac,<environment: 0x2b03270>


### Get the kipoi model and make a prediction for the example files

In [13]:
model <- kipoi$get_model("rbp_eclip/CSTF2")

In [14]:
predictions <- model$pipeline$predict_example()

In [15]:
predictions

0.27158695
0.09449492
0.09449492
0.27158695
0.27158695
0.27158695
0.09449492
0.27158695
0.27158695
0.27158695
0.27158695


In [16]:
class(predictions)

[1] "matrix"